<center>

<font size="5">MGMTMSA 403: Optimization</font>

</center>


<center>

<font size="5">Assignment 1: Operating Room Scheduling</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 11:59pm on January 24th.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
        Oscar Trumpy (706310371) <br>
        Facundo Klappenbach (306316186) <br>
    </div>
</div>


**<font size="5">Background</font>**

A hospital has six departments which require access to surgical facilities: General Surgery, Emergency, Neuro-surgery Opthamology, Oral Surgery, and Otolaryngology. Each department currently shares a set of three operating rooms, and is currently allocated the following amount of total operating room time (in hours per week):

Department | Total hrs/week of OR time | Time
--- | --- | ---
General Surgery | 92 | 48.4%
Emergency | 8 | 4.2%
Neurosurgery | 48 | 25.3%
Opthamology | 14 | 7.4%
Oral Surgery | 10 | 5.3%
Otolaryngology | 18 | 9.5%
Total | 190 | 100%

Next year, the hospital will be relocating to a larger hospital with five operating rooms. The new operating rooms will provide a total of 213.5 hours of operating room time per week (an increase from the current 190 hours per week). The availability of each room on each day of the week is given in the table below:

 . | Main-1 | Main-2 | Main-3 | Main-4 | Main-5
--- | --- | --- | --- | --- | ---
Monday | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-15:30
Tuesday | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-15:30
Wednesday | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-15:30
Thursday | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-17:00 | 8:00-15:30
Friday | 8:00-17:00 | 9:00-17:00 | 9:00-17:00 | 9:00-17:00 | 9:00-15:30

Moving to the new, larger hospital with more operating rooms means a new schedule must be devised. As a result, the hospital CEO has hired you to develop a new operating room schedule. The main concern of the CEO is to find a schedule that is equitable. Specifically, she wants the new schedule to allocate slots to each department such that each department maintains its current share (on a % basis) of the total operating room time.

**<font size="5">Questions</font>**

For each part below, (1) write the optimization model (you may do this by hand or in a text
editor), (2) include your Python code for the model, and (3) report the optimal value obtained at
an optimal solution.

**a)** First, formulate a schedule that minimizes the total under-allocation of each department to operating rooms. For example, General Surgery currently receives 48.4% of the total operating room time, and should therefore receive 48.4% of the total operating room time in the new schedule as well. In particular, the CEO has specified that under-allocation is to be avoided, meaning a penalty should be incurred if a department is allocated less than its target %, but there should be no penalty if a department is allocated more than its target % of operating room time.
Your model should produce a new schedule that minimizes the total under-allocation (on a percentage basis). The reason for representing allocation on a percentage basis is that using units of time is not equitable: For example, a loss of 1 hour per week is much more disruptive to Oral Surgery (currently 10 hrs/week) than it is for General Surgery (92 hrs/week). Note that time blocks cannot be split among multiple departments (e.g., if department i is assigned to Tuesday Main-2, then they receive the full 9 hours).

**HINT:** To get started, index departments using i = 1, . . . , 6, operating rooms using j = 1, . . . , 5, and days using k = 1, . . . , 5. Define the parameter t_i to represent the total number of target hours that department i should receive in the new schedule. Define the decision variable \s_i to represent the under-allocation of department i (i.e., in the optimal solution, s i should represent the total number of hours below the target t i that department i is allocated). The total under- allocation on a percentage basis can then be minimized using the following objective function:

\begin{align}
\text{minimize} \;\; & \sum_{i=1}^6 \frac{s_i}{t_i}
\end{align}

You are responsible for identifying the remaining decision variables and constraints for this problem.

**Solution a)**

\begin{align}
\text{minimize} \;\; & \sum_{i=1}^6 \frac{s_i}{t_i} \\
\text{s.t.} \;\; & \sum_{i=1}^6 X_{ijk} \le 1 \quad j = 1 \ldots 5, k = 1 \ldots 5\\
            & X_{ijk} \in \{0, 1 \} \quad i = 1 \ldots 6, j = 1 \ldots 5, k = 1 \ldots 5\\
            & hours_{k \times j} = 
            \begin{bmatrix}
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 8 & 8 & 8 & 6.5 \\
            \end{bmatrix}_{k \times j} \quad j = 1 \ldots 5, k = 1 \ldots 5 \\
            & allocation_i = \sum_{k=1}^5 \sum_{j=1}^5 X_{ijk} \times hours_{k \times j} \quad i = 1 \ldots 6 \\
            & target_{1 \times i} = 
            \begin{bmatrix}
            92 & 8 & 48 & 14 & 10 & 18\\
            \end{bmatrix}_{1 \times i} \times \frac{213.5}{190} \quad i = 1 \ldots 6 \\
            & s_i \ge target_i - allocation_i \quad i = 1 \ldots 6 \\
            & s_i \ge 0 \quad i = 1 \ldots 6 \\
\end{align}

In [303]:
#Importing necessary libraries
from gurobipy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import sys
import os
import random

In [304]:
##############################################################################################
# Define the data
# Defining the "hours per slot" matrix
hours = np.matrix([[9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 8, 8, 8, 6.5]])

# Defining the "target hours" array
target = np.array([92.0, 8.0, 48.0, 14.0, 10.0, 18.0])
target *= 213.5 / 190.0
target = target.round()

# Defining the names of departments, rooms, and days
dept = ['General Surgery', 'Emergency', 'Neurosurgery', 'Opthamology', 'Oral Surgery', 'Otolaryngology']
room = ['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5']
day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

##############################################################################################
# Initialize the model 
mod = Model()

##############################################################################################
# Define Variables 
# Defining binary "decision slot" variable
x = mod.addVars(len(dept), len(room), len(day), vtype = GRB.BINARY)

## Defining the "allocation hours" variable
allocation = mod.addVars(len(dept))
for i in range(len(dept)):
    allocation[i] = sum(x[i, j, k] * hours[k, j] for j in range(len(room)) for k in range(len(day)))

# Defining the "under-allocation" variable
s = mod.addVars(len(dept))

##############################################################################################
# Define Constraints
## Only one department can be assigned to a room on a day
for j in range(len(room)):
    for k in range(len(day)):
        mod.addConstr(sum(x[i, j, k] for i in range(len(dept))) <= 1)

## s_i as the under-allocation for department i
for i in range(len(dept)):
    mod.addConstr(s[i] >= target[i] - allocation[i])

## s_i >= 0
for i in range(len(dept)):
    mod.addConstr(s[i] >= 0)

##############################################################################################
# Construct Objective
mod.setObjective(sum((s[i] / target[i]) for i in range(len(dept))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod.update()
mod.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod.objVal)

##############################################################################################

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 162 columns and 312 nonzeros
Model fingerprint: 0xe62b7c46
Variable types: 12 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.01s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 0 continuous, 156 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 0.000000e+00, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00

In [305]:
# Generating tabular schedule
output = pd.DataFrame(columns = ['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5'], 
                      index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
allocated = [0, 0, 0, 0, 0, 0]

for j in range(len(room)):
    for k in range(len(day)):
        for i in range(len(dept)):
            if x[i, j, k].x == 1:
                allocated[i] += hours[k, j]
                output.iloc[k, j] = dept[i]

output

,Main 1,Main 2,Main 3,Main 4,Main 5
Monday,General Surgery,General Surgery,Neurosurgery,Neurosurgery,Otolaryngology
Tuesday,General Surgery,General Surgery,General Surgery,Neurosurgery,Otolaryngology
Wednesday,Neurosurgery,Neurosurgery,General Surgery,Neurosurgery,Otolaryngology
Thursday,General Surgery,General Surgery,Emergency,General Surgery,Oral Surgery
Friday,General Surgery,Opthamology,Opthamology,General Surgery,Oral Surgery


In [306]:
# Generating under-allocation analysis table
analysis = pd.DataFrame(columns = ['Target', 'Allocated', 'Under-Allocation', 'Ratio'], 
                        index = dept)

for i in range(len(dept)):
    analysis.iloc[i, 0] = target[i]
    analysis.iloc[i, 1] = allocated[i]
    analysis.iloc[i, 2] = target[i] - allocated[i]
    analysis.iloc[i, 3] = (target[i] - allocated[i]) / target[i]

analysis

,Target,Allocated,Under-Allocation,Ratio
General Surgery,103.0,98.0,5.0,0.048544
Emergency,9.0,9.0,0.0,0.0
Neurosurgery,54.0,54.0,0.0,0.0
Opthamology,16.0,16.0,0.0,0.0
Oral Surgery,11.0,14.0,-3.0,-0.272727
Otolaryngology,20.0,22.5,-2.5,-0.125


**b)** Operating rooms Main-1, Main-2 will be located on the first floor of the new hospital, Main-3 and Main-4 will be on the second floor, and Main-5 will be located on the third floor. To improve communication and mobility among department staff, the CEO has inquired whether it is possible to devise the schedule so that no department is split between two or more floors on the same day. For example, it is acceptable if a department is exclusively assigned to Main-1 on Monday and then Main-5 on Tuesday, but not acceptable if a department is assigned to Main-1 and Main-5 on the same day.
Incorporate constraints into your base model from part a) to ensure that no department is
allocated rooms on two different floors on the same day.
<br>

**HINT:** Index floors of the hospital using l = 1, 2, 3, and consider defining a new binary variable
to track which floor each department is assigned to.
<br>
<br>

**Solution b)**

\begin{align}
\text{minimize} \;\; & \sum_{i=1}^6 \frac{s_i}{t_i} \\
\text{s.t.} \;\; & \sum_{i=1}^6 X_{ijk} \le 1 \quad j = 1 \ldots 5, k = 1 \ldots 5\\
            & X_{ijk} \in \{0, 1 \} \quad i = 1 \ldots 6, j = 1 \ldots 5, k = 1 \ldots 5\\
            & hours_{k \times j} = 
            \begin{bmatrix}
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 9 & 9 & 9 & 7.5 \\
            9 & 8 & 8 & 8 & 6.5 \\
            \end{bmatrix}_{k \times j} \quad j = 1 \ldots 5, k = 1 \ldots 5 \\
            & allocation_i = \sum_{k=1}^5 \sum_{j=1}^5 X_{ijk} \times hours_{k \times j} \quad i = 1 \ldots 6 \\
            & target_{1 \times i} = 
            \begin{bmatrix}
            92 & 8 & 48 & 14 & 10 & 18\\
            \end{bmatrix}_{1 \times i} \times \frac{213.5}{190} \quad i = 1 \ldots 6 \\
            & s_i \ge target_i - allocation_i \quad i = 1 \ldots 6 \\
            & s_i \ge 0 \quad i = 1 \ldots 6 \\
\text{Additional Constraints} \;\; & \sum_{l=1}^3 Y_{ikl} \le 1 \quad i = 1 \ldots 6, k = 1 \ldots 5 \\
            & Y_{ikl} \in \{0, 1 \} \quad i = 1 \ldots 6, k = 1 \ldots 5, l = 1 \ldots 3 \\
            & floors_{j \times l} = 
            \begin{bmatrix}
            1 & 0 & 0 \\
            1 & 0 & 0 \\
            0 & 1 & 0 \\
            0 & 1 & 0 \\
            0 & 0 & 1 \\
            \end{bmatrix}_{j \times l} \quad j = 1 \ldots 5, l = 1 \ldots 3 \\
            & X_{ijk} \times floors_{jl} \le Y_{ikl}
\end{align}

In [307]:
##############################################################################################
# Define the data
# Defining the "hours per slot" matrix
hours = np.matrix([[9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 9, 9, 9, 7.5], [9, 8, 8, 8, 6.5]])

# Defining the "target hours" array
target = np.array([92.0, 8.0, 48.0, 14.0, 10.0, 18.0])
target *= 213.5 / 190.0
target = target.round()

# Defining the names of departments, rooms, and days
dept = ['General Surgery', 'Emergency', 'Neurosurgery', 'Opthamology', 'Oral Surgery', 'Otolaryngology']
room = ['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5']
day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

##############################################################################################
# Initialize the model 
mod = Model()

##############################################################################################
# Define Variables 
# Defining binary "decision slot" variable
x = mod.addVars(len(dept), len(room), len(day), vtype = GRB.BINARY)

## Defining the "allocation hours" variable
allocation = mod.addVars(len(dept))
for i in range(len(dept)):
    allocation[i] = sum(x[i, j, k] * hours[k, j] for j in range(len(room)) for k in range(len(day)))

# Defining the "under-allocation" variable
s = mod.addVars(len(dept))

##############################################################################################
# Additional Variables and Constraints for new model
# Define Data
floors = np.matrix([[1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1]])
level = ['First', 'Second', 'Third']

# Define Variables
y = mod.addVars(len(dept), len(day), len(level), vtype = GRB.BINARY)

# Define Constraints
for i in range(len(dept)):
    for k in range(len(day)):
        mod.addConstr(sum(y[i, k, l] for l in range(len(level))) <= 1)

for i in range(len(dept)):
    for j in range(len(room)):
        for k in range(len(day)):
            for l in range(len(level)):
                mod.addConstr(x[i, j, k] * floors[j, l] <= y[i, k, l])

##############################################################################################
# Define Constraints
## Only one department can be assigned to a room on a day
for j in range(len(room)):
    for k in range(len(day)):
        mod.addConstr(sum(x[i, j, k] for i in range(len(dept))) <= 1)

## s_i as the under-allocation for department i
for i in range(len(dept)):
    mod.addConstr(s[i] >= target[i] - allocation[i])

## s_i >= 0
for i in range(len(dept)):
    mod.addConstr(s[i] >= 0)

##############################################################################################
# Construct Objective
mod.setObjective(sum((s[i] / target[i]) for i in range(len(dept))), GRB.MINIMIZE)

##############################################################################################
# Update and solve
mod.update()
mod.optimize()

##############################################################################################
# Print Optimal Solution
print("Optimal Value:", mod.objVal)

##############################################################################################

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 517 rows, 252 columns and 1002 nonzeros
Model fingerprint: 0x378e824b
Variable types: 12 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Presolve removed 366 rows and 66 columns
Presolve time: 0.00s
Presolved: 151 rows, 186 columns, 606 nonzeros
Variable types: 0 continuous, 186 integer (180 binary)
Found heuristic solution: objective 5.8286263

Root relaxation: objective 1.359223e-01, 125 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0 

In [308]:
# Generating floor plan

plan = pd.DataFrame(columns = ['General Surgery', 'Emergency', 'Neurosurgery', 'Opthamology', 'Oral Surgery', 'Otolaryngology'],
                    index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])

for l in range(len(level)):
    for i in range(len(dept)):
        for k in range(len(day)):
            for j in range(len(room)):
                if (y[i, k, l].x == 1) & (x[i, j, k].x == 1):
                    plan.iloc[k, i] = level[l]   

plan = plan.fillna('')
plan

,General Surgery,Emergency,Neurosurgery,Opthamology,Oral Surgery,Otolaryngology
Monday,First,,Second,,,Third
Tuesday,Second,,First,Third,,First
Wednesday,Second,,First,First,,Third
Thursday,Second,Third,First,,,
Friday,First,Third,,,Second,


In [309]:
# Generating tabular schedule
output = pd.DataFrame(columns = ['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5'], 
                      index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
allocated = [0, 0, 0, 0, 0, 0]

for j in range(len(room)):
    for k in range(len(day)):
        for i in range(len(dept)):
            if x[i, j, k].x == 1:
                allocated[i] += hours[k, j]
                output.iloc[k, j] = dept[i]

output

,Main 1,Main 2,Main 3,Main 4,Main 5
Monday,General Surgery,General Surgery,Neurosurgery,Neurosurgery,Otolaryngology
Tuesday,Otolaryngology,Neurosurgery,General Surgery,General Surgery,Opthamology
Wednesday,Opthamology,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Thursday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Emergency
Friday,General Surgery,General Surgery,Oral Surgery,Oral Surgery,Emergency


In [310]:
# Generating under-allocation analysis table
analysis = pd.DataFrame(columns = ['Target', 'Allocated', 'Under-Allocation', 'Ratio'], 
                        index = dept)

for i in range(len(dept)):
    analysis.iloc[i, 0] = target[i]
    analysis.iloc[i, 1] = allocated[i]
    analysis.iloc[i, 2] = target[i] - allocated[i]
    analysis.iloc[i, 3] = (target[i] - allocated[i]) / target[i]

analysis

,Target,Allocated,Under-Allocation,Ratio
General Surgery,103.0,89.0,14.0,0.135922
Emergency,9.0,14.0,-5.0,-0.555556
Neurosurgery,54.0,54.0,0.0,0.0
Opthamology,16.0,16.5,-0.5,-0.03125
Oral Surgery,11.0,16.0,-5.0,-0.454545
Otolaryngology,20.0,24.0,-4.0,-0.2
